In [3]:
! pip install kaggle

In [5]:
#configuring the path of kaagle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.


cp: target `~/.kaggle/' is not a directory: No such file or directory
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
import json
with open('C:/Users/Pooja/OneDrive/Desktop/TSA/kaggle.json') as f:
    data = json.load(f)
data

{'username': 'priyapk97', 'key': 'ae53ed36bd72af06cd7a68cf6333099b'}

In [11]:
#API to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [12]:
#extract the compressed dataset from Zip file

from zipfile import ZipFile
dataset = 'C:/Users/Pooja/OneDrive/Desktop/TSA/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('Dataset Extracted')

Dataset Extracted


In [36]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')
from pprint import pprint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pooja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# pprint(stopwords.words('english'))

**DATA** **PROCESSING**

In [17]:
# twitter_data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = 'ISO-8859-1')
# twitter_data.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [9]:
#naming the column heading
cname = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv("training.1600000.processed.noemoticon.csv", names = cname, encoding = 'ISO-8859-1')
twitter_data.head(5)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
twitter_data.shape

(1600000, 6)

In [11]:
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [12]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [13]:
twitter_data.replace({'target':{4:1}}, inplace = True)

**STEMMING**
process of reducing word to its root word
Eg: actor,actress,acting = act

In [14]:
stemmer = PorterStemmer()

In [15]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [16]:
# twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)
# twitter_data.head(5)

KeyboardInterrupt: 

In [31]:
# df_to_csv = twitter_data.to_csv("stemmed_twitter.csv")

In [39]:
stemmed_df = pd.read_csv("stemmed_twitter_sc.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc3 in position 188141: invalid continuation byte

In [33]:
# stemmed_df.shape
print(stemmed_df.columns)

Index(['Unnamed: 0', 'target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')


In [30]:
#seperating data and column
X = stemmed_df['stemmed_content'].values
Y = stemmed_df['target'].values

KeyError: 'stemmed_content'

In [32]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values
print(X)
print(Y)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


Splitting the data into train & test

In [33]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.2, stratify = Y, random_state=2)

In [34]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


convert textual data to numerical as ML model wont understand these text split data

In [37]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

Training the Machine Learning Model - Logistic Regression(Classification Model)

In [38]:
model = LogisticRegression(max_iter=1000) 

In [39]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

MODEL EVALUVATION - Accuracy Score

In [40]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)
print("Accuracy on training data: ",training_data_accuracy)

Accuracy on training data:  0.79871953125


In [41]:
#accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)
print("Accuracy on test data: ",test_data_accuracy)

Accuracy on test data:  0.77668125


In [42]:
#saving the trained model
import pickle
filename = 'trained_model.pkl'
pickle.dump(model,open(filename, 'wb'))

use the saved model for future prediction

In [43]:
#loading saved model
load_model = pickle.load(open('trained_model.pkl', 'rb'))

In [52]:
#using exsisting train and test data the op is 1
X_new = X_test[3]
print("Actual Value = ",Y_test[3])
#lets test same with model if prediction is correct
pred = model.predict(X_new)
print("MODEL PREDICTED VALUE = ",pred)
print('Positive tweet') if(pred[0]==1) else print('Negative tweet')

Actual Value =  0
MODEL PREDICTED VALUE =  [0]
Negative tweet


In [6]:
%%writefile app.py
import streamlit as st
st.set_page_config(layout="wide")
st.title("TWITTER SENTIMENT ANALYSIS")
tweet = st.text_input("Enter the Tweet")

Overwriting app.py
